In [ ]:
# !nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#라이브러리 불러오기
import torch
import torch.nn as nn
import torch.optim as optim #학습 경사하강법
import torch.utils.data as data_utils #데이터 만들어주는 함수
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

In [ ]:
#데이터 불러오기
data_cov = pd.read_csv('/content/drive/MyDrive/딥러닝/DNN/covtype.csv')
data_cov['Cover_Type'].value_counts()
#class1 == 3, class0 == 4
data1= data_cov[data_cov['Cover_Type'] == 3]
data0 = data_cov[data_cov['Cover_Type'] == 4]

df = pd.concat([data1,data0])

dataset=df.sample(frac=1).reset_index(drop=True) #샘플 섞어주기 #데이터 섞어주기 frac은 비율이어서
dataset.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2111,298,31,90,63,212,120,212,224,582,...,0,0,0,0,0,0,0,0,0,3
1,2213,324,37,95,69,1211,107,176,195,1500,...,0,0,0,0,0,0,0,0,0,3
2,2016,302,26,30,12,60,140,218,214,765,...,0,0,0,0,0,0,0,0,0,3
3,2398,83,24,30,11,648,245,192,63,1110,...,0,0,0,0,0,0,0,0,0,3
4,2373,249,12,458,148,525,194,249,193,1771,...,0,0,0,0,0,0,0,0,0,3


In [ ]:
data_cov.shape #어마무시한 양

(581012, 55)

In [ ]:
#위의 코드 내가 이해하기 쉽게 변환
df = data_cov[(data_cov['Cover_Type'] == 3) | (data_cov['Cover_Type'] == 4)]

#이 방법은 처음 암
data = df.sample(frac = 1).reset_index(drop = True)
data

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2395,313,26,180,103,430,143,211,204,684,...,0,0,0,0,0,0,0,0,0,3
1,2488,141,16,60,-3,1663,242,234,117,797,...,0,0,0,0,0,0,0,0,0,3
2,2581,316,14,216,64,1914,182,227,185,700,...,0,0,0,0,0,0,0,0,0,3
3,2244,324,33,306,167,424,120,185,194,658,...,0,0,0,0,0,0,0,0,0,3
4,2260,310,13,0,0,1131,185,231,186,395,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496,2589,131,20,150,30,1506,247,226,98,2475,...,0,0,0,0,0,0,0,0,0,3
38497,2397,298,11,30,5,1080,188,236,188,1040,...,0,0,0,0,0,0,0,0,0,3
38498,2780,162,16,420,19,1605,234,242,135,1191,...,0,0,0,0,0,0,0,0,0,3
38499,2002,89,39,60,50,95,244,148,0,67,...,0,0,0,0,0,0,0,0,0,3


In [ ]:
print(df['Cover_Type'].value_counts()) #3의 비율이 압도적으로 높다.
print()
print(data['Cover_Type'].head(50))

3    35754
4     2747
Name: Cover_Type, dtype: int64

0     3
1     3
2     3
3     3
4     4
5     3
6     3
7     3
8     3
9     3
10    3
11    3
12    3
13    3
14    3
15    3
16    3
17    3
18    3
19    3
20    3
21    3
22    3
23    3
24    3
25    3
26    3
27    4
28    3
29    3
30    3
31    3
32    3
33    3
34    3
35    3
36    3
37    3
38    3
39    3
40    3
41    3
42    3
43    3
44    3
45    3
46    3
47    3
48    3
49    3
Name: Cover_Type, dtype: int64


In [ ]:
data_cov['Cover_Type'].value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: Cover_Type, dtype: int64

In [ ]:
dataset['Cover_Type'].replace([3, 4],[1, 0],inplace=True) #클래스의 이름 변경

In [ ]:
X = dataset.drop('Cover_Type', axis =1)
y = dataset['Cover_Type'] #x, y를 분리함.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

X_scaled = scaler.transform(X)

scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
scaled_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,-0.128946,0.828566,0.153583,1.488696,-1.063476,-1.252172,-1.038063,1.059085,1.528796,-0.370649,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.204278,0.125810,0.816671,-0.781408,-0.438200,1.630850,0.153858,1.131900,0.259537,0.936348,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.965112,-1.120411,-0.620020,0.049117,-0.015717,0.351728,0.575996,-0.360798,-0.574006,-0.357293,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.614828,1.203369,-0.951564,0.062960,-0.066415,-1.037320,-0.417271,0.695012,0.922583,-1.242616,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.044068,1.015968,-0.177961,0.256749,0.102579,-1.208869,-1.013231,0.767827,1.396187,-0.666393,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dataset = pd.concat([scaled_df,y],1) #1이 방향 아래로 or 옆으로

<ipython-input-8-c29dafd10400>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  dataset = pd.concat([scaled_df,y],1) #1이 방향 아래로 or 옆으로


In [ ]:
# 신규 (테스트) 데이터 생성 Train/ Validation /Test 아 테스트와 트레인 벨리데이션을 나눈
data = dataset.sample(frac=0.6, random_state=786) #0.6을 쓰겠다.
data_test = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_test.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_test.shape)) #훈련, 검증과 테스트데이터를 나누는

Data for Modeling: (23101, 55)
Unseen Data For Predictions: (15400, 55)


In [ ]:
print(data.shape)
print(data_test.shape)

(23101, 55)
(15400, 55)


In [ ]:
train, val = train_test_split(data, test_size = 0.3, random_state = 9492) #0.6벨리데이션을 나눈다

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
import pandas as pd

In [ ]:
print(train.shape)
print(val.shape)

(16170, 55)
(6931, 55)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #gpu를 사용할 수 있으면 사용하고, 아니면 cpu를 사용하겠다

train_x = train.iloc[:,:-1]
train_y = train.iloc[:,-1]
val_x = val.iloc[:,:-1]
val_y = val.iloc[:,-1]

trn_y  = torch.FloatTensor(train_y.values) #numpy를 텐서로 태운다
trn_X  = torch.FloatTensor(train_x.values)
val_y  = torch.FloatTensor(val_y.values)
val_X  = torch.FloatTensor(val_x.values)

In [ ]:
train = TensorDataset(trn_X, trn_y)
val = TensorDataset(val_X, val_y) #이걸로 학습에 사용되는 텐서 데이터 셋을 만든거야 하나로
#두개를 하나로 묶어준 텐서 데이터 셋`

In [ ]:
batch_size = 64
train_kwargs = {'dataset': train,
                'batch_size': batch_size,
               'shuffle': True}

test_kwargs = {'dataset': val,
                'batch_size': batch_size,
              'shuffle': False}
#배치 사이즈와 사용할 파라미터 넣어줌, 하나씩 넣어줘도 되는데, 이게 편해 딕셔너리로 #이게 데이터 로더의 파라미터

In [ ]:
#키워드 아그먼트 딕셔너리 구
train_loader = torch.utils.data.DataLoader(**train_kwargs)
#train_loader = torch.utils.data.DataLoader(data = train, ...)이런 역
val_loader  = torch.utils.data.DataLoader(**test_kwargs)

In [ ]:
from torch.utils.data import dataloader

In [ ]:
from sklearn.utils import shuffle
train_load = DataLoader(dataset= train, batch_size = batch_size, shuffle = True) #또 이렇게는 된다?

In [ ]:
train_load = DataLoader(**train_kwargs) #이것도 돼

In [ ]:
next(iter(train_load))

[tensor([[ 0.5936,  0.0790, -1.6147,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.3901,  0.1727,  0.5956,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.4461, -0.8955, -0.0674,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-1.2637,  1.6157,  2.1428,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.1585, -0.2958, -1.9462,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.2629, -0.2303, -0.7305,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 0., 1., 0., 1., 1., 1., 1., 1.])]

In [ ]:
print(next(iter(train_load))[0].shape)
print(next(iter(train_loader))[0].shape) #결국에는 이렇게도 똑같다.

torch.Size([64, 54])
torch.Size([64, 54])


In [ ]:
#gpu 사용
use_cuda = torch.cuda.is_available()
use_cuda

True

In [ ]:
#데이터 로더 for문으로 함께 동작

In [ ]:
next(iter(train_loader)) #마지막 데이터 로더 값을 넥스트 이터로 사용하면 된다.

[tensor([[-0.9177,  0.9410, -1.9462,  ...,  0.0000,  0.0000,  0.0000],
         [-0.4088, -0.5957, -0.7305,  ...,  0.0000,  0.0000,  0.0000],
         [-0.4597,  1.1940,  0.7062,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.4444, -0.5957,  0.2641,  ...,  0.0000,  0.0000,  0.0000],
         [-0.9380, -0.4645, -0.7305,  ...,  0.0000,  0.0000,  0.0000],
         [-1.2637,  0.5381, -0.0674,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
         1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 1., 0., 1.])]

In [ ]:
next(iter(train_loader))[0] #x에 데이터

tensor([[ 1.1941, -0.4926,  0.8167,  ...,  0.0000,  0.0000,  0.0000],
        [-0.7192, -1.1954, -0.2885,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0034, -1.0548, -0.3990,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-1.3451,  1.1753, -1.2831,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1289, -0.9424, -1.1726,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2171,  1.6719, -0.5095,  ...,  0.0000,  0.0000,  0.0000]])

In [ ]:
import torch.nn as nn

class DNN(nn.Module):

    def __init__(self, input_size, output_size, drop_rate):
      super(DNN, self).__init__()

      layer1 = [nn.Linear(input_size, 48),
              nn.BatchNorm1d(48),
              nn.Dropout(drop_rate),
              nn.ReLU]

      layer2 = [nn.Linear(48, 24),
              nn.BatchNorm1d(24),
              nn.Dropout(drop_rate),
              nn.ReLU()]

      layer3 = [nn.Linear(24, 12),
              nn.BatchNorm1d(12),
              nn.Dropout(drop_rate),
              nn.ReLU()]


      layer4 = [nn.Linear(12, output_size),
              nn.sigmoid()]

      self.first = nn.Sequential(*layer1)
      self.second = nn.Sequential(*layer2)
      self.third = nn.Sequential(*layer3)
      self.last = nn.Sequential(*layer4)


      def forward(self, x):
        x =  self.first(x)
        x =  self.second(x)
        x =  self.third(x)
        out = self.last(x)

In [ ]:
model.parameters

<bound method Module.parameters of DNN(
  (fisrt_layer): Sequential(
    (0): Linear(in_features=54, out_features=48, bias=True)
    (1): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (second_layer): Sequential(
    (0): Linear(in_features=48, out_features=24, bias=True)
    (1): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (third_layer): Sequential(
    (0): Linear(in_features=24, out_features=12, bias=True)
    (1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (last_layer): Sequential(
    (0): Linear(in_features=12, out_features=1, bias=True)
    (1): Sigmoid()
  )
)>

In [ ]:
class DNN(nn.Module): #객체 #nn.model 함수를 이미 짜놈 아까 텐서에서 nn.Modeul
#붕어빵 기계임
#여러개의 dnn을 만들 수 있따.
#붕어를 여러개 만들 수 있음
#붕어빵 만들 때 -> 팥, 밀가루 or 전기(self)
#input size: input ouput: 1 or 2
    def __init__(self, input_size, output_size, drop_rate): #이게 무조건 하는 있어야함 def_init(self) 붕어빵 만들때 들어가는 재료ㅣ 팥, 밀가루...
        super(DNN, self).__init__() #nn.Module 상속 받음 #droprate가 과적합 방지 , #얘도 항

        layer1 = [nn.Linear(input_size, 48), #wx+b를 해주는 함수 #이거 하나가 노드 한개 3개가 들어가면 48개가 나온다. 기껏했는데 수치가 커질수 있따 지나가면서.
                 nn.BatchNorm1d(48), #선형 변환 정규화
                 nn.Dropout(drop_rate), #과적합 방지, 0~1사이, 0.5의 확률로 비활성화 시키겠다. #하이퍼 파라미터
                 nn.ReLU()] #nn 아까 tensor flow 불러와서

        layer2 = [nn.Linear(48, 24),
                 nn.BatchNorm1d(24),
                 nn.Dropout(drop_rate),
                 nn.ReLU()]

        layer3 = [nn.Linear(24, 12),
                 nn.BatchNorm1d(12),
                 nn.Dropout(drop_rate),
                 nn.ReLU()]

        layer4 = [nn.Linear(12, output_size),
                 nn.Sigmoid()] #시그모이드 쓴다 일로 하려고.

        self.fisrt_layer = nn.Sequential(*layer1) #self 는 이름을 지어주는 거임.전기를 킨다 불을 킨다.
        self.second_layer = nn.Sequential(*layer2)
        self.third_layer = nn.Sequential(*layer3)
        self.last_layer = nn.Sequential(*layer4) #나중에 프린트 모델했을때 이름들이 뜸
        # Sequential 컨테이너 여기에 집어넣어주면 잘 조합됨. 리스트로 표현했는데
        #이것을 하기위해선 이 컨테이너 안에 담아줘야함.

    def forward(self, x):
        x = self.fisrt_layer(x)
        x = self.second_layer(x)
        x = self.third_layer(x)
        out = self.last_layer(x)

        return out #이 리턴 값

In [ ]:
DNN

__main__.DNN

In [ ]:
model_kwargs = {'drop_rate': 0.56,
              'input_size': 54,
               'output_size': 1}

training_kwargs = {'learning_rate': 1e-3,
              'epoch': 1}

model = DNN(**model_kwargs).to(device)

criterion = nn.BCELoss() #크로스엔트로피 binary cross entropy

optimizer = optim.Adam(model.parameters(), lr=training_kwargs['learning_rate']) #역전파 알고리즘등 계산을 해주는 것. 경사하강법을 하겠다.

num_epochs = training_kwargs['epoch']

In [ ]:
model.parameters #파라미터라는 소리다.

<bound method Module.parameters of DNN(
  (fisrt_layer): Sequential(
    (0): Linear(in_features=54, out_features=48, bias=True)
    (1): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (second_layer): Sequential(
    (0): Linear(in_features=48, out_features=24, bias=True)
    (1): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (third_layer): Sequential(
    (0): Linear(in_features=24, out_features=12, bias=True)
    (1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (last_layer): Sequential(
    (0): Linear(in_features=12, out_features=1, bias=True)
    (1): Sigmoid()
  )
)>

In [ ]:
print(model)

DNN(
  (fisrt_layer): Sequential(
    (0): Linear(in_features=54, out_features=48, bias=True)
    (1): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (second_layer): Sequential(
    (0): Linear(in_features=48, out_features=24, bias=True)
    (1): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (third_layer): Sequential(
    (0): Linear(in_features=24, out_features=12, bias=True)
    (1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (last_layer): Sequential(
    (0): Linear(in_features=12, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [ ]:
print(model) #레이어에 self를 붙여서 이름을 붙인거, 그냥 리스트는 순서가 없는데, 컨테이너에 넣으면서 순전파가 될 수 있음.

DNN(
  (fisrt_layer): Sequential(
    (0): Linear(in_features=54, out_features=48, bias=True)
    (1): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (second_layer): Sequential(
    (0): Linear(in_features=48, out_features=24, bias=True)
    (1): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (third_layer): Sequential(
    (0): Linear(in_features=24, out_features=12, bias=True)
    (1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (last_layer): Sequential(
    (0): Linear(in_features=12, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [ ]:
print('Neural network')
print(model)

Neural network
DNN(
  (fisrt_layer): Sequential(
    (0): Linear(in_features=54, out_features=48, bias=True)
    (1): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (second_layer): Sequential(
    (0): Linear(in_features=48, out_features=24, bias=True)
    (1): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (third_layer): Sequential(
    (0): Linear(in_features=24, out_features=12, bias=True)
    (1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (last_layer): Sequential(
    (0): Linear(in_features=12, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [ ]:
import copy
import warnings
warnings.filterwarnings("ignore") #불필요한 경고문을 안뜨게하려고.
#loss
criterion = nn.BCELoss()

def train(model, device, train_loader, optimizer, epoch):
    global criterion
    model.train() #?? batchnorm이랑 dropout-> 항상 활성화 된 상태 #붕어빵 하나.
    for batch_idx, (data, target) in enumerate(train_loader): #데이터 로더는 항상 for문이랑 동작을 한다고,
        data, target = data.to(device), target.to(device) #gpu상에서 계산하겠다.
        optimizer.zero_grad() #이건 지금 무시 #초기화한것이다.
        output = model(data) #output: 64 x 1 dnn 데이터를 집어넣음
        output = output.squeeze(1) #output: 64 #1차원으로 변환
        loss = criterion(output, target) #크로스 엔트로피를 계산한다. #배치 사이즈 , 64게의 크로스엔트로피
        loss.backward() #역전파 알고리즘.
        optimizer.step() #경사하강법 을 최적화 알고리즘을 따라서, 시작을 하겠다.
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item())) #100번째가 될때 loss하고 epoch를 출력하겠다.



def test():
    global criterion
    model.eval() # 배치놈, 드랍아웃이-> 비활성화 된 상태 model.eval은 학습을 하지 않는다는 상태.
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for batch_idx, (data, target) in enumerate (val_loader):
            valid_loss = {'loss_val' : []}

            data, target = data.to(device), target.to(device)
            outputs = model(data)
            outputs = outputs.squeeze()

            loss_val = criterion(outputs, target) #예측값이랑 실제값과 로스를 측정하고.
            valid_loss['loss_val'].append(loss_val.item()) #loss를 딕셔너리에 저장하고

            outputs = torch.flatten(outputs).cpu().detach().numpy() #cpu로 옮겨주고, detach를 뺀다. 그리고 넘파이로 바꾼다.
            target = torch.flatten(target).cpu().detach().numpy() #그냥 넘파이 어레이로 바꿔줌
            predicted = np.where(outputs>0.5, 1, 0) #0.5보다 크면 1 아니면 0

            n_samples += len(target)
            n_correct += (predicted == target).sum().item()
            vl_mean = np.array(valid_loss['loss_val']).mean()

            if batch_idx % 100 == 0:
                print('Batch mean Validation Loss: {:.6f}'.format(vl_mean))

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 validation set: {acc} %')
    return acc
    #정확도

In [ ]:
#OUTPUT_PATH = '/home/user/Code/JE/informs'
best_acc = -1 #하나도 못맞췄을때
best_model = None
for epoch in range(0,2): #한번만 돈다.
    train(model, device, train_loader, optimizer, epoch)
    acc = test()
    if acc > best_acc:
        best_acc = acc
        best_model = copy.deepcopy(model) #이 실행한 모델이 된다
        print(f'new best acc={best_acc}')
    else:
        print(f'current acc={acc}, prev_best_acc={best_acc}')

#state = {
#    'model_state': best_model.state_dict(),
#    'test_acc': best_acc
#}
#torch.save(state, OUTPUT_PATH+'/practice_nn.pt')  #훈련 끝나고 모델을 저장
print('best model saved')

Train Epoch: 0 [0/16170 (0%)]	Loss: 0.328033
Train Epoch: 0 [6400/16170 (40%)]	Loss: 0.218649
Train Epoch: 0 [12800/16170 (79%)]	Loss: 0.165441
Batch mean Validation Loss: 0.157777
Batch mean Validation Loss: 0.198249
Accuracy of the network on the 10000 validation set: 92.77160582888472 %
new best acc=92.77160582888472
Train Epoch: 1 [0/16170 (0%)]	Loss: 0.206448
Train Epoch: 1 [6400/16170 (40%)]	Loss: 0.326322
Train Epoch: 1 [12800/16170 (79%)]	Loss: 0.172387
Batch mean Validation Loss: 0.136087
Batch mean Validation Loss: 0.176361
Accuracy of the network on the 10000 validation set: 92.77160582888472 %
current acc=92.77160582888472, prev_best_acc=92.77160582888472
best model saved


In [ ]:
#new data inference
#테스트 데이터 만들어도 되는데 그냥 만듬
x = torch.normal(mean=6.9671e-06, std=0.4746, size=(1,54))
out = model(x.to(device))
out = torch.flatten(out).cpu().detach().numpy()
predicted = np.where(out>0.5, 1, 0)

In [ ]:
x

tensor([[ 0.3255, -0.2420, -0.0176, -0.0218, -0.3425,  0.1255,  0.2266,  0.2175,
          0.0089, -0.1959,  0.5173,  0.4480, -0.5324, -0.9759,  0.1711,  0.3284,
         -0.2987, -0.5580, -0.3406,  0.1503, -0.5544,  0.0501,  0.0644,  0.4655,
          0.4563,  0.4013, -0.4562,  0.2566, -0.3195,  0.3382, -0.0459,  0.1190,
         -0.1617,  0.5039, -0.9309,  0.1608,  0.4193, -0.1102, -0.5001, -0.1837,
          0.2015,  0.2417, -0.1312, -0.3441,  0.0629, -1.0008, -0.2442,  0.1018,
          0.3050,  0.5124,  0.2053,  0.2270,  0.9484, -0.2251]])

In [ ]:
print('예측 클래스:', predicted)

예측 클래스: [1]


## [HW] 3-layer neural network 를 구현하고 학습하세요(batch norm, drop out, activation function 자유 구성)

In [ ]:
class 3_layer_nn(nn.Module):

    def __init__(self):
        super(3_layer_nn, self).__init__()

        '''여기에 코드를 작성하세요'''

    def forward(self, x):
        '''여기에 코드를 작성하세요'''

        return